**Adaptive Learning: Deaths**

In [34]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from itertools import product
import warnings
warnings.filterwarnings("ignore")

In [35]:
class ModelGroupSpecs:
    def __init__(self, ar_orders, desired_model_groups):
        self.ar_orders = ar_orders
        self.desired_model_groups = desired_model_groups

    def get_all_possible_combinations(self, model_group, MG_ar_orders, MG_regressors):
        return list(product(model_group, MG_ar_orders, MG_regressors))

    def create_functional_sets(self):
        output = []
        if "AR" in self.desired_model_groups:
            # MG1 = Model Group 1: AR models.

            self.AR_models = self.get_all_possible_combinations(
                model_group=['AR'],
                MG_ar_orders=self.ar_orders,
                MG_regressors=[None])
            output.append(self.AR_models)

        if "OLS" in self.desired_model_groups:
            # MG2N = Model Group 2N: Single Variable Exogenous OLS

            self.OLS_models = self.get_all_possible_combinations(
                model_group=['OLS'],
                MG_ar_orders=[None],#'log_new_vaccines_per_capita	', 
                MG_regressors=['log_new_people_vaccinated_per_capita','delta_cases_per_capita',
                                'delta_cases_per_capita_United_Kingdom', 'delta_cases_per_capita_Germany', 'delta_cases_per_capita_France',
                                'delta_deaths_per_capita_United Kingdom', 'delta_deaths_per_capita_Germany', 'delta_deaths_per_capita_France',
                                'full_lockdown', 'full_lockdown.l30', 'full_lockdown.l45',
                                'max_tp', 'min_tp', 'rain', 'humidity',
                                'day_of_the_week',  'trend'])

            output.append(self.OLS_models)

        if "ARX" in self.desired_model_groups:
            # MG2T and MG3T: Introducing lagged dependent terms to the previous model specifications.

            self.ARX_models = self.get_all_possible_combinations(
                model_group=['ARX'],
                MG_ar_orders=self.ar_orders,
                MG_regressors=['log_new_people_vaccinated_per_capita',  'delta_cases_per_capita',
                                'delta_deaths_per_capita_United Kingdom', 'delta_deaths_per_capita_Germany', 'delta_deaths_per_capita_France',
                                'full_lockdown', 'full_lockdown.l30', 'full_lockdown.l45',
                                'max_tp', 'min_tp', 'rain', 'humidity',
                                'day_of_the_week', 'trend'])
                                
            output.append(self.ARX_models)

        # Returning the functional sets to be deployed.
        return output

In [36]:
ar_orders = np.arange(1, 3)

In [37]:
models = ModelGroupSpecs(ar_orders, ['AR', "OLS", "ARX"])

In [38]:
model_groups = models.create_functional_sets()

In [39]:
model_groups

[[('AR', 1, None), ('AR', 2, None)],
 [('OLS', None, 'log_new_people_vaccinated_per_capita'),
  ('OLS', None, 'delta_cases_per_capita'),
  ('OLS', None, 'delta_cases_per_capita_United_Kingdom'),
  ('OLS', None, 'delta_cases_per_capita_Germany'),
  ('OLS', None, 'delta_cases_per_capita_France'),
  ('OLS', None, 'delta_deaths_per_capita_United Kingdom'),
  ('OLS', None, 'delta_deaths_per_capita_Germany'),
  ('OLS', None, 'delta_deaths_per_capita_France'),
  ('OLS', None, 'full_lockdown'),
  ('OLS', None, 'full_lockdown.l30'),
  ('OLS', None, 'full_lockdown.l45'),
  ('OLS', None, 'max_tp'),
  ('OLS', None, 'min_tp'),
  ('OLS', None, 'rain'),
  ('OLS', None, 'humidity'),
  ('OLS', None, 'day_of_the_week'),
  ('OLS', None, 'trend')],
 [('ARX', 1, 'log_new_people_vaccinated_per_capita'),
  ('ARX', 1, 'delta_cases_per_capita'),
  ('ARX', 1, 'delta_deaths_per_capita_United Kingdom'),
  ('ARX', 1, 'delta_deaths_per_capita_Germany'),
  ('ARX', 1, 'delta_deaths_per_capita_France'),
  ('ARX', 1, '

In [40]:
combined = pd.read_csv("Data/Combined_Dataset.csv")

In [41]:
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
import statsmodels.api as sm

def training_and_forecasting(k,
                data_and_time,
                y_var,
                functional_sets,
                ):
    """
    Input specifications (by example):
        forecast_horizon=3, known as $k$ in our main work.
        Other choices include 1, 2, 5, 10.
        data_and_time. This should be a pandas dataframe including (1) a time index, (2) the dependent variable of interest
        and (3) exogenous variables.
        functional_sets. This is the set of all models to be trained. It is obtained via the ModelGroupSpecs class.
        ar_orders = [1, 2,..., 10]. For MG2T and MG3T these order run to up to 5.
        window_sizes = [22, 63, 126, 252].

    Return:
        - The set of forecasts produced by all fixed model groups.
    """

    # Step 1 (a): Defining our indexes for training and testing.
    T_max = len(data_and_time) - 1
    T_train = np.arange(100, T_max+1)
    # Logistics
    # Creation of H_tilda

    forecast_df = create_forecast_df(functional_sets)

    # Step 2 (a): Training the models and making a forecast.
    for t in tqdm(T_train, leave=True, position=0):

        forecasts_t = [t]

        for functional_set in functional_sets:
            for model_group, ar_order, regressor in functional_set:

                w = t-2

                if model_group == 'AR':

                    forecasts = train_and_forecast_MG1(data = data_and_time[y_var],
                                                       t = t,
                                                       w =w,
                                                       ar_order = ar_order,
                                                       k = k)

                elif model_group == 'OLS':

                    forecasts = train_and_forecast_MG2N(regressor = data_and_time[regressor],
                                                        dep_data = data_and_time[y_var],
                                                        t = t,
                                                        w = w,
                                                        k = k)

                elif model_group == 'ARX':

                    forecasts = train_and_forecast_MG_2T_3T(dep_data=data_and_time[y_var],
                                                            regressor=data_and_time[regressor],
                                                            t=t,
                                                            w = w,
                                                            ar_order=ar_order,
                                                            k=k)


        
                forecasts_t.append(float(forecasts))
        
        forecast_df.loc[t] = forecasts_t

    return forecast_df


##### Individual Training and forecasting Functions See Appendix 1.5 and 1.6 of the pseudo-algorithm #####

def train_and_forecast_MG1(data, t, w, ar_order, k):

    windowed_data = np.array(data[(t - w + 1): (t + 1)])

    model = ARIMA(windowed_data, order=(
        ar_order, 0, 0), trend = 'c').fit()

    forecasts = model.forecast(k)[::-1]

    return forecasts[-1]


def train_and_forecast_MG2N(regressor, dep_data, t, w, k):

    forecasts = []

    indep_train_data = np.array(regressor[max(0, t - w - k + 1): (t - k + 1)])

    indep_train_data = sm.add_constant(indep_train_data)

    dep_train_data = np.array(dep_data[max(0, t - w + 1): (t + 1)])

    model = sm.OLS(dep_train_data,
                   indep_train_data).fit()

    for i in range(k):

        forecasts.append(model.params[0] + model.params[1] * regressor[t - i])

    return forecasts[-1]


def train_and_forecast_MG_2T_3T(dep_data, regressor, t, w, ar_order, k):

    indep_train_data = vectorise_indep_variables(dep_to_be_lagged = dep_data,
                                                 exog=regressor,
                                                 t=t,
                                                 ar_order=ar_order,
                                                 w=w,
                                                 k=k)

    dep_train_data = dep_data[max(0, t - w + 1): (t + 1)]

    model = sm.OLS(dep_train_data, sm.add_constant(indep_train_data)).fit()

    forecasts = forecast_with_lags(
        model.params, dep_data, regressor, t, ar_order, k)

    return forecasts


def vectorise_indep_variables(dep_to_be_lagged, exog, t, ar_order, w, k):

    lagged_dep_data = list(dep_to_be_lagged[(t - w - ar_order): (t)])

    lagged_dep_data = lagged_dep_data[::-1]

    lagged_p = [lagged_dep_data[j: j+ar_order]
                for j in range(0, len(lagged_dep_data) - ar_order)]

    lagged_p = np.array(lagged_p[:: -1])

    exog = np.array(exog[(t - w - k + 1): (t - k + 1)])

    vectorised_training_data = np.array(
        [np.append(lagged_p[j], [exog[j]]) for j in range(0, len(exog))])

    return vectorised_training_data


def forecast_with_lags(model_params, dep_to_be_lagged, regressor, t, ar_order, k):

    forecasts = []

    lagged_y = np.array(
        dep_to_be_lagged[(t - ar_order + 1): (t + 1)])
    lagged_x = np.array(regressor[(t - k + 1): (t + 1)])

    for tau in range(0, k):  # tau = 0 means 1-step-ahead.

        observations_under_consideration = [[1]]

        if len(forecasts) > 0:
            observations_under_consideration.append(
                forecasts[::-1][:ar_order])

        observations_under_consideration.append(lagged_y[tau:][::-1])

        observations_under_consideration.append(lagged_x[tau].flatten())

        observations_under_consideration = np.concatenate(
            observations_under_consideration).ravel()

        forecasts.append(np.dot(np.array(observations_under_consideration),
                                np.array(model_params)))

    return forecasts[-1]


#### Helper Functions ####

def naming(model_group, ar_order, regressor):
    return f"{model_group}, AR{ar_order}, Regressor = {regressor}"

def create_forecast_df(functional_sets):
    headers = ['t']
    for functional_set in functional_sets:
        for model_group, ar_order, regressor in functional_set:
            headers.append(naming(model_group, ar_order, regressor))
    forecast_df = pd.DataFrame(columns=[headers])
    return forecast_df


In [43]:
forecasts = training_and_forecasting(1, combined, "delta_deaths_per_capita", model_groups)

100%|██████████| 511/511 [04:32<00:00,  1.88it/s]


In [44]:
del forecasts['t']

In [45]:
import numpy as np
from pandas import read_csv as rc
from tqdm import tqdm
from itertools import product

def adaptive_learning(k,
                      data_and_time,
                      forecast_df,
                      y_var,
                      functional_sets,
                      specification_learning
                      ):

    # Loading
    AL_specification = specification_learning[0]

    if AL_specification == 'EN':
        v, p, Lambda = specification_learning[1]
        lambda_vector = [Lambda**(v-i) for i in range(0, v)]

    # Step 1: Housekeeping.
    # Step 1 (a):
    T_max = len(data_and_time) - 1

    # Step 1 (b): Define the training index.
    T_train = np.arange(100,
                        T_max)
                        

    # Step 1 (c): Define the testing index.
    T_test = np.arange(100 + v,
                       T_max)

    forecast_errors = create_H_tilda_dict(functional_sets)

    # Step 2.
    for t in tqdm(T_train, leave=True, position=0):
        for model_name in functional_sets:

            # Step 2 (a)(i): Obtain a forecast according to the model.

            forecast_value = forecast_df.loc[t, model_name]

        
            # Step 2 (a)(ii): Obtain the forecasting error.
            val_data = data_and_time.loc[t+k, y_var]

            error_t_plus_k = abs(forecast_value - val_data)

            forecast_errors[model_name][t+k] = error_t_plus_k

    # Step 3. Implement AL via the designated option.

    if AL_specification == "EN":
        optimal_models = {}
        y_star_dict = {}

    # Step 3(c).
    for t in tqdm(T_test, leave=True, position=0):

        # Step 3 (c)(i): Declare T_tilda (the adaptive learning lookback window).
        T_tilda = np.arange(t - v + 1, t + 1)

        if AL_specification == "EN":

            y_star, h_star = regular_AL(T_tilda=T_tilda,
                                        t=t,
                                        AL_specification="EN",
                                        functional_sets=functional_sets,
                                        forecast_df=forecast_df,
                                        forecast_errors=forecast_errors,
                                        lambda_vector=lambda_vector,
                                        p=p)



            # Step 3 (d): Save this h star (best model) and make an associated forecast
            optimal_models.update({t: h_star})
            y_star_dict.update({t: y_star})


        Output = [
            functional_sets,
            [T_test, T_train],
            optimal_models,
            y_star_dict
        ]

    return Output


def regular_AL(T_tilda,
               t,
               AL_specification,
               functional_sets,
               forecast_df,
               forecast_errors,
               lambda_vector,
               p):

    # Step 3 (c)(ii).
    loss_by_model = {}

    for model_name in functional_sets:

        # Step 3 (c)(ii)(A): Collect the array of forecasting errors
        # over the adaptive learning lookback window and evaluate the loss over the period T tilda.
        errors = [forecast_errors[model_name][tau] for tau in T_tilda]

        if AL_specification == "EN":
            loss = exponential_learning(errors=errors, lam=lambda_vector, p=p)

        loss_by_model.update({model_name: loss})

    # Step 3 (c)(ii)(B): Find the argmin of the loss function for h in H over the period Tilda.
    h_star = min(loss_by_model, key=loss_by_model.get)

    # Step 3 (c)(ii)(C): Save this h star (best model) and make the associated forecast.
    y_star = forecast_df.loc[t, h_star]

    return float(y_star), h_star


###### 1.7.1 Exponential-Norm Learning Function ######

def exponential_learning(errors, lam, p):
    e_vector = np.array(errors)
    lambda_vector = np.array(lam)
    loss = np.dot(np.squeeze(np.power(e_vector, p)), lambda_vector)
    return loss

##### Helper Functions ####


def create_H_tilda_dict(H):
    H_tilda = {}
    for model in H:
        H_tilda.update({model: {}})
    return H_tilda


def create_value_dict(H):
    H_tilda = {}
    for model in H:
        H_tilda.update({model: 0})
    return H_tilda


In [55]:
AL = adaptive_learning(1, 
                       combined, 
                       forecast_df = forecasts, 
                       y_var = 'delta_deaths_per_capita', 
                       functional_sets = forecasts.columns, 
                       specification_learning= ["EN", [100, 2, 0.96]])

100%|██████████| 410/410 [00:01<00:00, 333.21it/s]


In [56]:
preds = pd.DataFrame(AL[3], index = [i for i in AL[3].keys()]).iloc[0, :]

In [57]:
pd.DataFrame(list(AL[2].values())).value_counts()

OLS, ARNone, Regressor = delta_deaths_per_capita_United Kingdom    151
ARX, AR1, Regressor = delta_deaths_per_capita_United Kingdom        79
ARX, AR1, Regressor = delta_deaths_per_capita_France                40
OLS, ARNone, Regressor = full_lockdown                              22
ARX, AR2, Regressor = day_of_the_week                               20
ARX, AR2, Regressor = delta_deaths_per_capita_United Kingdom        17
ARX, AR1, Regressor = day_of_the_week                               16
AR, AR2, Regressor = None                                           14
ARX, AR1, Regressor = full_lockdown.l45                             12
OLS, ARNone, Regressor = delta_deaths_per_capita_France             11
ARX, AR2, Regressor = delta_deaths_per_capita_France                 9
AR, AR1, Regressor = None                                            6
ARX, AR2, Regressor = full_lockdown.l45                              4
ARX, AR1, Regressor = full_lockdown                                  4
ARX, A

In [58]:
first_half = pd.DataFrame(list(AL[2].values())).loc[:len(list(AL[2].values()))/2]
second_half = pd.DataFrame(list(AL[2].values())).loc[len(list(AL[2].values()))/2:]

In [59]:
first_half.value_counts().to_latex("first_half.txt")

In [60]:
first_half.value_counts()

OLS, ARNone, Regressor = delta_deaths_per_capita_United Kingdom    48
ARX, AR1, Regressor = delta_deaths_per_capita_France               35
ARX, AR1, Regressor = delta_deaths_per_capita_United Kingdom       29
ARX, AR2, Regressor = day_of_the_week                              20
ARX, AR1, Regressor = day_of_the_week                              16
AR, AR2, Regressor = None                                          14
ARX, AR1, Regressor = full_lockdown.l45                            12
ARX, AR2, Regressor = delta_deaths_per_capita_France                9
AR, AR1, Regressor = None                                           6
ARX, AR1, Regressor = full_lockdown                                 4
ARX, AR2, Regressor = delta_deaths_per_capita_United Kingdom        4
ARX, AR2, Regressor = full_lockdown.l45                             4
ARX, AR1, Regressor = humidity                                      2
ARX, AR1, Regressor = max_tp                                        1
ARX, AR2, Regressor 

In [61]:
second_half.value_counts().to_latex("second_half.txt")

In [62]:
second_half.value_counts()

OLS, ARNone, Regressor = delta_deaths_per_capita_United Kingdom    103
ARX, AR1, Regressor = delta_deaths_per_capita_United Kingdom        51
OLS, ARNone, Regressor = full_lockdown                              22
ARX, AR2, Regressor = delta_deaths_per_capita_United Kingdom        13
OLS, ARNone, Regressor = delta_deaths_per_capita_France             11
ARX, AR1, Regressor = delta_deaths_per_capita_France                 5
dtype: int64